In [1]:
import os
import random
import shutil
import cv2
import numpy as np

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation, Flatten, Dropout, Dense
from keras.layers.normalization import BatchNormalization
import matplotlib.image as mpimg
from sklearn.preprocessing import LabelBinarizer

Using TensorFlow backend.


In [2]:
proportion = 0.1
BATCH_SIZE = 10
SIZE = 150
default_image_size = tuple((128,128))

In [ ]:
# Get train and val data
data_train = os.listdir('tomato/train_lite/')
data_val = os.listdir('tomato/test_lite/')

In [ ]:
# Get a proportion of whole data
for folder in data_train:
    i = 0
    for data in os.listdir('tomato/val/'+folder):
        if(i < proportion * len(os.listdir('tomato/val/'+folder))):
            shutil.copyfile('tomato/val/'+folder+'/'+data, 'tomato/test_lite/'+folder+'/'+data)
            i += 1

In [3]:
base_dir = 'tomato'

train_dir = 'tomato/train_lite'
test_dir = 'tomato/test_lite'

In [4]:
# All images will be rescaled by 1./255.
train_datagen = ImageDataGenerator( rescale = 1.0/255. )
test_datagen  = ImageDataGenerator( rescale = 1.0/255. )


train_generator = train_datagen.flow_from_directory(train_dir,
                                                    batch_size=BATCH_SIZE,
                                                    class_mode='categorical',
                                                    target_size=(150, 150))     

test_generator =  test_datagen.flow_from_directory(test_dir,
                                                         batch_size=BATCH_SIZE,
                                                         class_mode  = 'categorical',
                                                         target_size = (150, 150))

Found 1000 images belonging to 10 classes.
Found 100 images belonging to 10 classes.


In [5]:
#chanDim = -1
model = tf.keras.models.Sequential([
    # Note the input shape is the desired size of the image 150x150 with 3 bytes color
    tf.keras.layers.Conv2D(128, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(3,3),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2), 
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'), 
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'), 
    tf.keras.layers.MaxPooling2D(2,2),
    # Flatten the results to feed into a DNN
    tf.keras.layers.Flatten(), 
    # 512 neuron hidden layer
    tf.keras.layers.Dense(1024, activation='relu'), 
    # Only 1 output neuron. It will contain a value from 0-1 where 0 for 1 class ('cats') and 1 for the other ('dogs')
    tf.keras.layers.Dense(10, activation='softmax')  
])

In [6]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 148, 148, 128)     3584      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 49, 49, 128)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 47, 47, 128)       147584    
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 45, 45, 128)       147584    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 22, 22, 128)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 20, 20, 128)       147584    
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 18, 18, 128)       1

In [7]:
model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss='categorical_crossentropy',
              metrics = ['accuracy'])

In [8]:
#aug = ImageDataGenerator(fill_mode="nearest")
history = model.fit(train_generator,
                              validation_data=test_generator,
                              steps_per_epoch=100,
                              epochs=50,
                              verbose=1)
# history = model.fit(train_generator,
#                               validation_data=test_generator,
#                               steps_per_epoch=100,
#                               epochs=50,
#                               verbose=1)

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 100 steps, validate for 10 steps
Epoch 1/50
100/100 [==============================] - 69s 694ms/step - loss: 2.1898 - accuracy: 0.1490 - val_loss: 2.0367 - val_accuracy: 0.1600
Epoch 2/50
100/100 [==============================] - 44s 442ms/step - loss: 1.8052 - accuracy: 0.3160 - val_loss: 1.5710 - val_accuracy: 0.4400
Epoch 3/50
100/100 [==============================] - 46s 462ms/step - loss: 1.5672 - accuracy: 0.4270 - val_loss: 1.4444 - val_accuracy: 0.4800
Epoch 4/50
100/100 [==============================] - 46s 458ms/step - loss: 1.4122 - accuracy: 0.4910 - val_loss: 1.3431 - val_accuracy: 0.5500
Epoch 5/50
100/100 [==============================] - 43s 433ms/step - loss: 1.1989 - accuracy: 0.5620 - val_loss: 1.2706 - val_accuracy: 0.5600
Epoch 6/50
100/100 [==============================] - 44s 443ms/step - loss: 1.1325 - accuracy: 0.6060 - val_loss: 1.4068 - val_accuracy: 0.5200
Epoch 7/50
100/100 [=================

KeyboardInterrupt: 

In [ ]:
random.shuffle(data_train)

In [ ]:
print(data_train)